In [1]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>  蓝色的 | 好看的 | 小小的
"""

In [2]:
another_grammer = """
#
"""

In [4]:
import random

In [4]:
def adj():  return random.choice('蓝色的 | 好看的 | 小小的'.split('|') ).split()[0]

In [5]:
def adj_star():
    return random.choice([lambda : '', lambda : adj() + adj_star()])()

In [6]:
adj_star()

'好看的好看的'

# 设计自己的语言生成器

In [1]:
shop_service = """
shop = 尊称 自我介绍 时间节点 推荐内容
尊称 = 称谓 你好, 
称谓 = 老伯 | 阿姨 | 小姐 | 小朋友 | 先生 | 女生
自我介绍 = 我是 职务 姓名 。
职务 = 柜员 | 促销员 | 店长
姓名 = 小张 | 小王 | 小李
时间节点 = 现在是 时间 ,
时间 = 早上 | 中午 | 晚上 | 下午
推荐内容 = 请问您 行为动词 商品名 吗？
行为动词 = 想买 | 是在找 
商品名 = 啤酒 | 果汁 | 三明治 | 饼干 | 棒棒糖
"""

In [2]:
def create_grammer(grammer_str, split='=>', line_split='\n'):
    grammer = {}
    for line in grammer_str.split(line_split):
        if not line.strip(): continue
        exp, stmt = line.split(split)
        grammer[exp.strip()] = [i.split() for i in stmt.split('|')]
    return grammer

In [5]:
choice = random.choice

def generate(gram, target):
    if target not in gram: return target # 达到最细粒度了，target is a terminal expression
    
    expand = [generate(gram, t) for t in choice(gram[target])]
    return ''.join(e for e in expand if e != 'null')

In [6]:
for i in range(10):
    print(generate(gram=create_grammer(shop_service, split='=', line_split='\n'), target='shop'))

老伯你好,我是促销员小李。现在是下午,请问您是在找啤酒吗？
老伯你好,我是柜员小张。现在是早上,请问您想买啤酒吗？
先生你好,我是柜员小张。现在是晚上,请问您是在找三明治吗？
老伯你好,我是促销员小李。现在是中午,请问您是在找饼干吗？
女生你好,我是促销员小王。现在是中午,请问您是在找啤酒吗？
女生你好,我是促销员小李。现在是晚上,请问您想买三明治吗？
阿姨你好,我是柜员小王。现在是中午,请问您想买三明治吗？
女生你好,我是促销员小张。现在是下午,请问您想买三明治吗？
阿姨你好,我是店长小李。现在是下午,请问您是在找果汁吗？
老伯你好,我是柜员小张。现在是晚上,请问您想买棒棒糖吗？


In [8]:
def generate_n(n: int):
    sentences = []
    for i in range(n):
        senten = generate(gram=create_grammer(shop_service, split='=', line_split='\n'), target='shop')
        sentences.append(senten)
    return sentences

In [9]:
generate_n(10)

['先生你好,我是促销员小李。现在是早上,请问您想买啤酒吗？',
 '女生你好,我是店长小张。现在是中午,请问您想买饼干吗？',
 '老伯你好,我是柜员小张。现在是下午,请问您想买棒棒糖吗？',
 '小姐你好,我是店长小张。现在是晚上,请问您想买棒棒糖吗？',
 '老伯你好,我是促销员小李。现在是中午,请问您想买三明治吗？',
 '小姐你好,我是促销员小张。现在是下午,请问您是在找三明治吗？',
 '阿姨你好,我是柜员小王。现在是早上,请问您是在找啤酒吗？',
 '阿姨你好,我是促销员小王。现在是晚上,请问您是在找棒棒糖吗？',
 '老伯你好,我是柜员小李。现在是下午,请问您想买果汁吗？',
 '阿姨你好,我是促销员小王。现在是下午,请问您想买啤酒吗？']

# 使用新数据源完成新语言模型

In [10]:
csv_file_path = '/Users/Administrator/Downloads/movie_comments.csv'

In [11]:
import pandas as pd
import jieba
import re
from collections import Counter

In [12]:
content = pd.read_csv(csv_file_path, encoding='utf-8', low_memory=False)

In [13]:
content[90:100]

,id,link,name,comment,star
90,91,https://movie.douban.com/subject/26363254/,战狼2,#电影院#一个人就买了四张票（不打折）请全家看，心疼我的小钱钱≥﹏≤ 还好家长们的反应是好的...,4
91,92,https://movie.douban.com/subject/26363254/,战狼2,听说过夜郎自大吗？我第一次知道中国人这么牛逼。中国部队所向披靡，连特么坦克都能给你开漂移。整...,2
92,93,https://movie.douban.com/subject/26363254/,战狼2,我跟你们讲，吴京当年在电视上大吹牛逼，说自己在杀破狼里和甄子丹真打，多厉害多牛逼。结果花絮里...,1
93,94,https://movie.douban.com/subject/26363254/,战狼2,「中國狼不咬中國人！」www,1
94,95,https://movie.douban.com/subject/26363254/,战狼2,冷锋像是一个符号，他代表着千千万万守护我们的军人。“国强则民安”，生活在这个没有战争的国度，...,5
95,96,https://movie.douban.com/subject/26363254/,战狼2,非常难看 也不知道导演哪儿来的自信,1
96,97,https://movie.douban.com/subject/26363254/,战狼2,一部政治宣传片！中国也就在非洲还有点脸面了！给宏大的战争场面和吴京卖力的打斗五星，剧情减一星...,3
97,98,https://movie.douban.com/subject/26363254/,战狼2,诚意满满，全程无尿点。吴京非常帅，剧情比战狼1好看多了。,5
98,99,https://movie.douban.com/subject/26363254/,战狼2,23点45分开始，1点48分结束，我看了12次手机，影厅的天花板上有8条线组合成三角形装饰，...,2
99,100,https://movie.douban.com/subject/26363254/,战狼2,有一场坦克戏，简直令人浮想连连啊，吴京真的不是故意的嘛。又是压人压成肉泥的镜头，又是吴京站在...,4


In [14]:
comments = content['comment'].tolist()

In [15]:
def token(string):
    return ''.join(re.findall('\w+', string))

In [16]:
comments_clean = [token(str(a)) for a in comments]

In [17]:
def cut(sentence):
    return list(jieba.cut(sentence)) if sentence else []

In [18]:
def get_token():
    TOKEN = []  
    for i in comments_clean:
        TOKEN.extend(cut(i))
    return TOKEN

In [20]:
TOKEN = get_token()

In [21]:
comment_count = Counter(TOKEN)

In [22]:
comment_count.most_common(10)

[('的', 328262),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065),
 ('和', 31290)]

In [23]:
TOKEN[:20]

['吴京',
 '意淫',
 '到',
 '了',
 '脑残',
 '的',
 '地步',
 '看',
 '了',
 '恶心',
 '想',
 '吐',
 '首映礼',
 '看',
 '的',
 '太',
 '恐怖',
 '了',
 '这个',
 '电影']

In [24]:
def prob_1(word):
    return comment_count[word] / len(TOKEN)

In [25]:
comment_count['吴京']

278

In [26]:
TOKEN_2_GRAM = [TOKEN[i] + TOKEN[i+1] for i  in range(len(TOKEN))[:-2]]

In [27]:
comment_count_2 = Counter(TOKEN_2_GRAM)

In [28]:

comment_count_2.most_common(10)

[('的电影', 8640),
 ('看的', 7106),
 ('都是', 6335),
 ('让人', 5284),
 ('的故事', 4709),
 ('看了', 4585),
 ('也是', 4408),
 ('的时候', 4398),
 ('的人', 4356),
 ('的是', 4348)]

In [29]:
def prob_2(word1, word2):
    # 2-gram language_module
    if word1 + word2 in comment_count_2:
        return comment_count_2[word1 + word2] / comment_count[word1]
    else:
        return 1 / len(TOKEN_2_GRAM)

In [30]:
prob_2('电影', '恐怖')

2.969561989606533e-05

In [35]:
ssenten_test = generate_n(10)

In [36]:
ssenten_test

['女生你好,我是促销员小王。现在是中午,请问您是在找棒棒糖吗？',
 '先生你好,我是柜员小李。现在是下午,请问您想买饼干吗？',
 '女生你好,我是促销员小王。现在是晚上,请问您想买棒棒糖吗？',
 '先生你好,我是促销员小王。现在是晚上,请问您是在找啤酒吗？',
 '小朋友你好,我是柜员小王。现在是下午,请问您是在找啤酒吗？',
 '先生你好,我是促销员小王。现在是早上,请问您是在找三明治吗？',
 '老伯你好,我是店长小张。现在是晚上,请问您是在找饼干吗？',
 '阿姨你好,我是店长小王。现在是下午,请问您想买棒棒糖吗？',
 '先生你好,我是店长小张。现在是早上,请问您是在找饼干吗？',
 '阿姨你好,我是店长小李。现在是晚上,请问您想买三明治吗？']

In [37]:
def get_sentence_probability(sentence):
    words = cut(sentence)
    
    sentence_prob = 1
    for i, word in enumerate(words[:-1]):
        word_next = words[i+1]
        prob = prob_2(word, word_next)
        sentence_prob *= prob
    return sentence_prob

In [48]:
def generate_best_sentence(grammer=shop_service, n=5):
    result = []
    for i in range(n):
        sentence = generate(gram=create_grammer(shop_service, split='=', line_split='\n'), target='shop')
        prob = get_sentence_probability(sentence)
        result.append((sentence, prob))
    result = sorted(result, key=lambda x: x[1], reverse=True)
    for item in result:
        print(f'sentence: {item[0]} with probability {item[1]}')
    print(f'The best sentence is {result[0][0]}')

In [49]:
get_sentence_probability('阿姨你好,我是店长小李。现在是晚上,请问您想买三明治吗？')

4.315929374692288e-101

In [50]:
generate_best_sentence(n=5)

sentence: 小姐你好,我是店长小张。现在是下午,请问您是在找饼干吗？ with probability 1.1349919435852813e-97
sentence: 老伯你好,我是店长小张。现在是早上,请问您想买饼干吗？ with probability 5.301853513091648e-99
sentence: 老伯你好,我是柜员小李。现在是晚上,请问您想买三明治吗？ with probability 4.315929374692288e-101
sentence: 小姐你好,我是柜员小张。现在是下午,请问您想买饼干吗？ with probability 4.315929374692288e-101
sentence: 阿姨你好,我是促销员小张。现在是晚上,请问您想买饼干吗？ with probability 4.315929374692288e-101
The best sentence is 小姐你好,我是店长小张。现在是下午,请问您是在找饼干吗？
